In [ ]:
%%time
# for development, changes in other modules have to be reloaded to reflect changes
%load_ext autoreload
%autoreload 1

import pandas as pd 
import numpy as np 
from matplotlib import pyplot as plt 
import seaborn as sbn 
import os, sys
import SimpleITK as sitk

%aimport config 
sys.path.append(config.lib_dir)
%aimport utils
%aimport segment 
%aimport match 
%aimport register 
%aimport evaluate 
%aimport qc 
sys.path.append(config.script_dir)
from slide_registration import *

# Overview

The goal of this notebook is to compare the efficacy of the full slide's registrations for a given experiment.


In [ ]:
class args:
    def __init__(self):
        args.input = ['/mnt/e/CycIF_analysis/registration_outputs/test_restitched/']
        args.slide = ['D1']
        args.scene = ['None']
        
args = args()

In [ ]:
img_file_names = [x for x in os.listdir(args.input[0]) if x[-4:] == '.tif']
parsed_names = pd.DataFrame([parse_file_name(x) for x in img_file_names])

# filter to slide/scene
parsed_names = parsed_names[lambda x: (x.slide_name == args.slide[0]) & (x.scene == args.scene[0])]

# grab only the dapi channel 
parsed_names = parsed_names[lambda x: x.color_channel == 'c1']

parsed_names.head()

In [ ]:
imgs_reg = utils.load_imgs_mt(parsed_names.original.values, '/mnt/e/CycIF_analysis/registration_outputs/test_restitched/')

In [ ]:
imgs_unreg = utils.load_imgs_mt(parsed_names.original.values, '/mnt/z/Marilyne/Axioscan/6-Dhivya/split/No_Scene/')

# Testing de-dusting alg

In [ ]:
gaussian = sitk.DiscreteGaussianImageFilter()
gaussian.SetVariance( (10, 10) )

?gaussian

In [ ]:
def dedust(im): 
    '''

    '''

    # load image 
    #im = sitk.ReadImage(args.input[0] + '/' + info.original, sitk.sitkUInt8)

    # gaussian blur to avoid grabbing random high intesnity pixels 
    gaussian = sitk.DiscreteGaussianImageFilter()
    gaussian.SetVariance( (100, 100) )
    gaussian.SetMaximumKernelWidth(1000)
    im_blur  = gaussian.Execute ( im )
    utils.myshow(im_blur[::10, ::10], figsize=(10,10))
    
    # create mask image
    # should really use quantile or something... 
    thresh = np.quantile(sitk.GetArrayFromImage(im).flatten(), 0.99)
    print(thresh)
    mask = im_blur > thresh #100
    
    utils.myshow(mask[::10, ::10])

    # create mask filter 
    filt = sitk.MaskImageFilter()
    filt.SetMaskingValue(1)
    filt.SetOutsideValue(0)

    # modify image
    im2 = filt.Execute(image=im, maskImage=mask) 

    # rescale image 
    im2 = sitk.RescaleIntensity(im2, outputMinimum=0, outputMaximum=255)

    # save image
    #sitk.WriteImage(im2, args.output[0] + '/' + info.original)

    utils.myshow(im2[::10,::10], figsize=(10,10))

In [ ]:
for _im in imgs_unreg:
    print(_im)
    im = imgs_unreg[_im]
    dedust(im)

# Evaluating slide registration

In [ ]:
fixed = imgs_reg['R0_AF488.AF555.AF647.AF750_D1_2020_09_30__9123_c1_ORG.tif']
utils.myshow(fixed[::10, ::10])
seg_fixed = fixed > 10
seg_fixed = sitk.Cast(seg_fixed, sitk.sitkFloat32)
utils.myshow(seg_fixed[::10, ::10])


In [ ]:
sitk.GetArrayFromImage(fixed).max()

In [ ]:
gaussian = sitk.DiscreteGaussianImageFilter()
gaussian.SetVariance( 10 )
fixed_blur  = gaussian.Execute ( fixed )

In [ ]:
seg_fixed = fixed_blur > np.quantile(sitk.GetArrayFromImage(fixed_blur), 0.75)

In [ ]:
utils.myshow(seg_fixed[::10, ::10])

In [ ]:
seg_fixed = sitk.Cast(seg_fixed, sitk.sitkInt8)

In [ ]:
def plot_reg(moving, fixed, ax, title): 
    
    #moving  = gaussian.Execute ( moving )
    seg_fixed = fixed > np.quantile(fixed, 0.75)
    seg_fixed = sitk.Cast(fixed, sitk.sitkInt8)
    seg_moving = moving > np.quantile(sitk.GetArrayFromImage(moving), 0.75)
    seg_moving = sitk.Cast(seg_moving, sitk.sitkInt8)
    
    sub = seg_fixed - seg_moving
    utils.myshow(sub[::10, ::10], ax=ax, title=title)

for _im in imgs_reg: 
    print(_im)
    moving_reg = imgs_reg[_im]
    #moving_unreg = imgs_unreg[_im]
    
    #moving_unreg = sitk.Resample(moving_unreg, seg_fixed)
    #moving_reg = sitk.Resample(moving_reg, fixed)#seg_fixed)
    
    f, axes = plt.subplots(1,1, figsize=(12,12))
    plot_reg(moving_reg, fixed, axes, 'registered')
    
    #plot_reg(moving_unreg, seg_fixed, axes.flat[1], 'unregistered')
    plt.show()